# Download data

In [ ]:
!curl -s https://d1bjgq97if6urz.cloudfront.net/Public/Peilingwijzer/Last/Cijfers_Peilingwijzer.xlsx -o Cijfers_Peilingwijzer.xlsx

# Import data

In [ ]:
import pandas as pd

In [ ]:
numbers = pd.read_excel('Cijfers_Peilingwijzer.xlsx', header=0, index_col=0, engine="openpyxl")

## Sanity check: recreate plot from Peilingwijzer
https://peilingwijzer.tomlouwerse.nl/p/laatste-cijfers.html

In [ ]:
zetels = numbers.drop(columns=['Datum', 'Percentage', 'PercentageLaag', 'PercentageHoog'])

In [ ]:
zetels['ZetelsHoog'] = zetels['ZetelsHoog'] - zetels['Zetels']
zetels['Zetels'] = zetels['Zetels'] - zetels['ZetelsLaag']

In [ ]:
zetels = zetels.reindex(['ZetelsLaag', 'Zetels', 'ZetelsHoog'], axis=1)

In [ ]:
zetels.plot.bar(stacked=True, figsize=(11, 8))

In [ ]:
numbers.Zetels['VVD']

# Coalitiewijzer!
Press the buttons and get the number of seats of your selected coalition. A majority coalition needs 76 seats.

In [ ]:
import ipywidgets as widgets
from ipyvuetify import VuetifyTemplate, Btn
import traitlets

In [ ]:
class SeatsWidget(VuetifyTemplate):
    rows = traitlets.Int(10).tag(sync=True)
    cols = traitlets.Int(15).tag(sync=True)
    low = traitlets.Int(0).tag(sync=True)
    exp = traitlets.Int(0).tag(sync=True)
    high = traitlets.Int(0).tag(sync=True)
    template = traitlets.Unicode('''
<template>
  <div>
  <table>
  <tr v-for="i in rows">
    <td v-for="j in cols">
    <div>
      <v-icon v-if="(j + i * cols) < low">mdi-seat</v-icon>
      <v-icon v-else-if="(j + i * cols) < exp"
              style="color: orange">mdi-seat</v-icon>
      <v-icon v-else-if="(j + i * cols) == exp" style="color: green">mdi-seat</v-icon>
      <v-icon v-else-if="(j + i * cols) < high" style="color: red">mdi-seat</v-icon>
      <v-icon v-else style="color: pink">mdi-seat</v-icon>
    </div>
    </td>
  </tr>
  </div>
</template>
''').tag(sync=True)    
seats = SeatsWidget()

In [ ]:
toggle_buttons = {key: widgets.ToggleButton(description=key, value=False,
                                            layout=widgets.Layout(height='auto', width='auto'))
                  for key in numbers.index}

grid_width = 5

grid = widgets.GridspecLayout((len(numbers) - 1) // grid_width + 1, grid_width)
for ix, (key, button) in enumerate(toggle_buttons.items()):
    ix1, ix2 = ix // grid_width, ix % grid_width
    grid[ix1, ix2] = button

ASCII_grid_width = 30    

ASCII_grid_height = (150 - 1) // ASCII_grid_width + 1
ASCII = widgets.GridspecLayout(ASCII_grid_height, ASCII_grid_width)

for ix in range(150):
    ix1, ix2 = ix // ASCII_grid_width, ix % ASCII_grid_width

    ASCII[ix1, ix2] = widgets.HTML(value='.', disabled=False,
                                   layout=widgets.Layout(height='auto', width='auto'),
                                   )

In [ ]:
def on_toggle(**toggles):
    expectation_value = int(numbers[list(toggles.values())].Zetels.sum())
    low = int(numbers[list(toggles.values())].ZetelsLaag.sum())
    high = int(numbers[list(toggles.values())].ZetelsHoog.sum())
#     print(f"verwacht aantal zetels: {expectation_value} (laag: {low}, hoog: {high})")

    seats.low = low
    seats.exp = expectation_value
    seats.high = high

    count = 1
    for ix in range(150):
        ix1, ix2 = ix % ASCII_grid_height, ix // ASCII_grid_height
        if count > high:
            value = '.'
        elif count > expectation_value:
            value = 'o'
        elif count == expectation_value:
            value = 'O'
        elif count >= low:
            value = 'o'
        else:
            value = '-'
        count += 1
        
        ASCII[ix1, ix2].value = value

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(grid, interact_out)
# display(ASCII)
display(seats)